# NBA Player Performance Dynamics: Player Stability Analysis

This notebook builds on the dynamical systems analysis from the previous notebook to classify players into distinct stability profiles, examine case studies of different player types, and translate our mathematical findings into practical basketball insights.

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
from datetime import datetime
from sklearn.utils import resample

# Add the project root to the path so we can import our modules
sys.path.append('..')

# Import our modules
from src.dynamics import (
    calculate_var_model,
    calculate_stability_metrics,
    plot_stability_distribution,
    calculate_performance_prediction,
    decompose_performance_factors,
    plot_performance_decomposition
)
from src.visualization import create_player_stability_quadrant
from src.utils import setup_plotting_style

# Set up plotting style
setup_plotting_style()

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

## Load Player Dynamics Data

Let's load the player dynamics data that we calculated in the previous notebook.

In [ ]:
# Load the player dynamics data
try:
    player_analysis = pd.read_csv('../data/processed/player_dynamics.csv')
    player_temporal_df = pd.read_csv('../data/processed/player_temporal.csv')
    
    # Convert date strings to datetime objects
    player_temporal_df['GAME_DATE'] = pd.to_datetime(player_temporal_df['GAME_DATE'])
    
    print(f"Loaded player dynamics data with {len(player_analysis)} players")
    print(f"Loaded player temporal data with {len(player_temporal_df)} records")
except FileNotFoundError:
    print("Processed data not found. Please run the 02_dynamical_systems.ipynb notebook first.")

In [ ]:
# Examine the player dynamics data
player_analysis.head()

## Player Classification Framework

Based on our stability metrics, we can classify players into distinct categories that provide insights into their performance characteristics.

### Define Stability Classifications

We classify players into five stability categories based on their system stability score:

1. **Highly Stable** (system_stability < 0.7): Consistently reliable performance with minimal game-to-game variation
2. **Stable** (0.7 ≤ system_stability < 0.9): Generally consistent performance with occasional variation
3. **Moderately Stable** (0.9 ≤ system_stability < 1.1): Balanced mix of consistency and variability
4. **Volatile** (1.1 ≤ system_stability < 1.3): Significant game-to-game variation in performance
5. **Chaotic** (system_stability ≥ 1.3): Highly unpredictable performance with extreme variation

These categories help teams understand the reliability and predictability of player performance.

In [ ]:
# Display the number of players in each stability category
stability_type_counts = player_analysis['stability_type'].value_counts().reset_index()
stability_type_counts.columns = ['Stability Type', 'Count']
stability_type_counts['Percentage'] = stability_type_counts['Count'] / len(player_analysis) * 100

# Sort by stability level
stability_order = ['Highly Stable', 'Stable', 'Moderately Stable', 'Volatile', 'Chaotic']
stability_type_counts['Stability Type'] = pd.Categorical(stability_type_counts['Stability Type'], categories=stability_order, ordered=True)
stability_type_counts = stability_type_counts.sort_values('Stability Type')

stability_type_counts

In [ ]:
# Visualize the stability categories
plt.figure(figsize=(12, 6))
sns.barplot(x='Stability Type', y='Count', data=stability_type_counts, palette='viridis')
plt.xlabel('Stability Type', fontsize=12)
plt.ylabel('Number of Players', fontsize=12)
plt.title('Distribution of Player Stability Types', fontsize=14)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Find examples of players in each stability category
stability_examples = {}

for stability_type in stability_order:
    # Get players with this stability type
    stability_players = player_analysis[player_analysis['stability_type'] == stability_type]
    
    if len(stability_players) > 0:
        # Sort by points
        examples = stability_players.nlargest(5, 'avg_pts')
        stability_examples[stability_type] = examples[['player_name', 'avg_pts', 'avg_plus_minus', 'system_stability']]

# Display examples for each stability type
for stability_type, examples in stability_examples.items():
    print(f"\n{stability_type} Examples:")
    print(examples)

### Create Quadrant Visualization

We can create a quadrant visualization that plots system stability against plus/minus to identify four key player types:

1. **High-Value Stability** (Lower Left): Consistently positive impact players
2. **High-Ceiling Volatility** (Upper Right): Players with game-changing ability but inconsistent output
3. **Low-Impact Consistency** (Lower Left): Reliably average performers
4. **High-Risk Variability** (Upper Right): Unpredictable performers with negative tendencies

In [ ]:
# Create player stability quadrant visualization
fig = create_player_stability_quadrant(player_analysis)
plt.show()

In [ ]:
# Define the quadrants
stability_threshold = 1.0
impact_threshold = 0

player_analysis['quadrant'] = 'Unknown'
player_analysis.loc[(player_analysis['system_stability'] < stability_threshold) & 
                    (player_analysis['avg_plus_minus'] > impact_threshold), 'quadrant'] = 'High-Value Stability'
player_analysis.loc[(player_analysis['system_stability'] >= stability_threshold) & 
                    (player_analysis['avg_plus_minus'] > impact_threshold), 'quadrant'] = 'High-Ceiling Volatility'
player_analysis.loc[(player_analysis['system_stability'] < stability_threshold) & 
                    (player_analysis['avg_plus_minus'] <= impact_threshold), 'quadrant'] = 'Low-Impact Consistency'
player_analysis.loc[(player_analysis['system_stability'] >= stability_threshold) & 
                    (player_analysis['avg_plus_minus'] <= impact_threshold), 'quadrant'] = 'High-Risk Variability'

# Count players in each quadrant
quadrant_counts = player_analysis['quadrant'].value_counts().reset_index()
quadrant_counts.columns = ['Quadrant', 'Count']
quadrant_counts['Percentage'] = quadrant_counts['Count'] / len(player_analysis) * 100

quadrant_counts

In [ ]:
# Visualize the quadrant distribution
plt.figure(figsize=(10, 6))
sns.barplot(x='Quadrant', y='Count', data=quadrant_counts, palette='viridis')
plt.xlabel('Quadrant', fontsize=12)
plt.ylabel('Number of Players', fontsize=12)
plt.title('Distribution of Player Quadrants', fontsize=14)
plt.xticks(rotation=45)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Compare player statistics across different quadrants
quadrant_stats = player_analysis.groupby('quadrant').agg({
    'avg_pts': 'mean',
    'avg_plus_minus': 'mean',
    'system_stability': 'mean',
    'performance_entropy': 'mean',
    'pts_volatility': 'mean',
    'player_id': 'count'
}).reset_index()

# Rename count column
quadrant_stats = quadrant_stats.rename(columns={'player_id': 'player_count'})

# Sort by player count
quadrant_stats = quadrant_stats.sort_values('player_count', ascending=False)

quadrant_stats

In [ ]:
# Visualize the quadrant statistics
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Points by quadrant
sns.barplot(x='quadrant', y='avg_pts', data=player_analysis, ax=axes[0, 0])
axes[0, 0].set_title('Average Points by Quadrant', fontsize=12)
axes[0, 0].set_xlabel('')
axes[0, 0].tick_params(axis='x', rotation=45)

# Plus/Minus by quadrant
sns.barplot(x='quadrant', y='avg_plus_minus', data=player_analysis, ax=axes[0, 1])
axes[0, 1].set_title('Average Plus/Minus by Quadrant', fontsize=12)
axes[0, 1].set_xlabel('')
axes[0, 1].tick_params(axis='x', rotation=45)

# Stability by quadrant
sns.barplot(x='quadrant', y='system_stability', data=player_analysis, ax=axes[1, 0])
axes[1, 0].set_title('System Stability by Quadrant', fontsize=12)
axes[1, 0].set_xlabel('')
axes[1, 0].tick_params(axis='x', rotation=45)

# Entropy by quadrant
sns.barplot(x='quadrant', y='performance_entropy', data=player_analysis, ax=axes[1, 1])
axes[1, 1].set_title('Performance Entropy by Quadrant', fontsize=12)
axes[1, 1].set_xlabel('')
axes[1, 1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

## Case Studies of Different Player Types

Let's examine specific players from each quadrant to understand the basketball implications of their stability profiles.

In [ ]:
# Find examples of players in each quadrant
quadrant_examples = {}

for quadrant in player_analysis['quadrant'].unique():
    # Get players in this quadrant
    quadrant_players = player_analysis[player_analysis['quadrant'] == quadrant]
    
    # Sort by points for high-value and high-ceiling quadrants
    if quadrant in ['High-Value Stability', 'High-Ceiling Volatility']:
        examples = quadrant_players.nlargest(3, 'avg_pts')
    else:
        # Sort by absolute plus/minus for low-impact and high-risk quadrants
        examples = quadrant_players.nlargest(3, 'avg_plus_minus')
    
    quadrant_examples[quadrant] = examples[['player_name', 'avg_pts', 'avg_plus_minus', 'system_stability', 'stability_type']]

# Display examples for each quadrant
for quadrant, examples in quadrant_examples.items():
    print(f"\n{quadrant} Examples:")
    print(examples)

In [ ]:
# Select a player from each quadrant for detailed analysis
case_study_players = {}

for quadrant, examples in quadrant_examples.items():
    if len(examples) > 0:
        case_study_players[quadrant] = examples.iloc[0]['player_name']

# Display selected players
case_study_players

In [ ]:
# Analyze game-to-game performance for case study players
for quadrant, player_name in case_study_players.items():
    # Get player data
    player_id = player_analysis[player_analysis['player_name'] == player_name]['player_id'].iloc[0]
    player_data = player_temporal_df[player_temporal_df['Player_ID'] == player_id].copy()
    player_data = player_data.sort_values('GAME_DATE')
    
    # Plot points and plus/minus
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 10), sharex=True)
    
    # Points
    ax1.plot(player_data['GAME_DATE'], player_data['PTS'], marker='o', linestyle='-', label='Points')
    ax1.plot(player_data['GAME_DATE'], player_data['PTS_MA5'], linestyle='-', color='red', label='5-Game Moving Avg')
    ax1.set_ylabel('Points', fontsize=12)
    ax1.set_title(f"{player_name}: {quadrant}", fontsize=14)
    ax1.grid(True, alpha=0.3)
    ax1.legend()
    
    # Plus/Minus
    ax2.plot(player_data['GAME_DATE'], player_data['PLUS_MINUS'], marker='o', linestyle='-', color='green', label='Plus/Minus')
    ax2.plot(player_data['GAME_DATE'], player_data['Performance_Momentum'], linestyle='-', color='purple', label='Performance Momentum')
    ax2.set_xlabel('Game Date', fontsize=12)
    ax2.set_ylabel('Plus/Minus', fontsize=12)
    ax2.grid(True, alpha=0.3)
    ax2.axhline(y=0, color='gray', linestyle='--')
    ax2.legend()
    
    plt.tight_layout()
    plt.show()
    
    # Calculate and display key statistics
    pts_mean = player_data['PTS'].mean()
    pts_std = player_data['PTS'].std()
    pts_cv = pts_std / pts_mean if pts_mean > 0 else 0
    pm_mean = player_data['PLUS_MINUS'].mean()
    pm_std = player_data['PLUS_MINUS'].std()
    
    print(f"\n{player_name} ({quadrant}) Statistics:")
    print(f"Points: {pts_mean:.1f} ± {pts_std:.1f} (CV: {pts_cv:.2f})")
    print(f"Plus/Minus: {pm_mean:.1f} ± {pm_std:.1f}")
    print(f"System Stability: {player_analysis[player_analysis['player_name'] == player_name]['system_stability'].iloc[0]:.2f}")
    print(f"Performance Entropy: {player_analysis[player_analysis['player_name'] == player_name]['performance_entropy'].iloc[0]:.2f}")
    print("\n" + "-"*50)

In [ ]:
# Analyze point distributions for case study players
plt.figure(figsize=(14, 10))

for i, (quadrant, player_name) in enumerate(case_study_players.items()):
    # Get player data
    player_id = player_analysis[player_analysis['player_name'] == player_name]['player_id'].iloc[0]
    player_data = player_temporal_df[player_temporal_df['Player_ID'] == player_id].copy()
    
    # Create subplot
    plt.subplot(2, 2, i+1)
    
    # Plot points distribution
    sns.histplot(player_data['PTS'], kde=True)
    plt.axvline(x=player_data['PTS'].mean(), color='red', linestyle='--', label='Mean')
    plt.title(f"{player_name}: {quadrant}")
    plt.xlabel('Points')
    plt.ylabel('Frequency')
    plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
# Analyze autocorrelation of points for case study players
from pandas.plotting import autocorrelation_plot

plt.figure(figsize=(14, 10))

for i, (quadrant, player_name) in enumerate(case_study_players.items()):
    # Get player data
    player_id = player_analysis[player_analysis['player_name'] == player_name]['player_id'].iloc[0]
    player_data = player_temporal_df[player_temporal_df['Player_ID'] == player_id].copy()
    player_data = player_data.sort_values('GAME_DATE')
    
    # Create subplot
    plt.subplot(2, 2, i+1)
    
    # Plot autocorrelation
    autocorrelation_plot(player_data['PTS'])
    plt.title(f"{player_name}: {quadrant}")
    plt.ylim(-1, 1)

plt.tight_layout()
plt.show()

## Basketball Interpretation

Now let's translate our mathematical findings into basketball insights.

### Performance Implications of Different Stability Types

Based on our analysis, we can draw several basketball insights about the performance implications of different stability profiles:

1. **High-Value Stability Players**:
   - Provide consistent positive impact game after game
   - Serve as reliable cornerstones for team success
   - Perform well in high-pressure situations due to predictability
   - Ideal for building team identity and culture
   - Examples: [Player examples from your data]

2. **High-Ceiling Volatility Players**:
   - Capable of exceptional performances that can win games single-handedly
   - Also prone to significant down games that can hurt the team
   - Require strategic management to maximize upside while minimizing downside
   - Best used in specific matchups or situations where their strengths can be leveraged
   - Examples: [Player examples from your data]

3. **Low-Impact Consistency Players**:
   - Provide reliable but limited contributions
   - Valuable as role players who understand their limitations
   - Can be effective in specific situations or matchups
   - May be undervalued in traditional evaluation frameworks
   - Examples: [Player examples from your data]

4. **High-Risk Variability Players**:
   - Unpredictable performance with negative tendencies
   - Can disrupt team chemistry and game plans
   - Require significant development focus on consistency
   - May be overvalued based on occasional standout performances
   - Examples: [Player examples from your data]

In [ ]:
# Update the basketball interpretation with actual player examples
for quadrant, examples in quadrant_examples.items():
    print(f"\n{quadrant} Examples:")
    for i, row in examples.iterrows():
        print(f"- {row['player_name']}: {row['avg_pts']:.1f} PPG, {row['avg_plus_minus']:.1f} +/-, {row['system_stability']:.2f} Stability")

### Strategic Applications

Our stability analysis has several strategic applications for basketball operations:

1. **Player Acquisition**:
   - Target high-value stability players for core roster positions
   - Add high-ceiling volatility players for specific roles
   - Avoid high-risk variability players unless available at significant discount
   - Consider low-impact consistency players for specific role player needs

2. **Lineup Construction**:
   - Balance lineups with a mix of stability and volatility
   - Pair high-ceiling volatility players with high-value stability players
   - Avoid lineups with multiple high-risk variability players
   - Use low-impact consistency players in specific matchup situations

3. **Game Strategy**:
   - Rely on high-value stability players in high-leverage situations
   - Deploy high-ceiling volatility players strategically based on matchups
   - Limit minutes for high-risk variability players in close games
   - Use low-impact consistency players in specific defensive or role situations

4. **Player Development**:
   - Focus on reducing volatility for high-ceiling volatility players
   - Work on consistency and decision-making for high-risk variability players
   - Help low-impact consistency players expand their impact areas
   - Maintain the stability of high-value stability players

## Correlation with Traditional Metrics

Let's examine how our stability metrics correlate with traditional basketball statistics.

In [ ]:
# Calculate additional traditional stats for each player
player_traditional_stats = player_temporal_df.groupby('Player_ID').agg({
    'PlayerName': 'first',
    'PTS': ['mean', 'std'],
    'REB': ['mean', 'std'],
    'AST': ['mean', 'std'],
    'STL': ['mean', 'std'],
    'BLK': ['mean', 'std'],
    'TOV': ['mean', 'std'],
    'PLUS_MINUS': ['mean', 'std'],
    'MIN': ['mean', 'std'],
    'Game_ID': 'count'
}).reset_index()

# Flatten the column names
player_traditional_stats.columns = ['_'.join(col).strip('_') for col in player_traditional_stats.columns.values]

# Rename some columns for clarity
player_traditional_stats = player_traditional_stats.rename(columns={
    'PlayerName_first': 'player_name',
    'Game_ID_count': 'games_played'
})

# Calculate coefficient of variation (CV) for key stats
player_traditional_stats['PTS_cv'] = player_traditional_stats['PTS_std'] / player_traditional_stats['PTS_mean']
player_traditional_stats['REB_cv'] = player_traditional_stats['REB_std'] / player_traditional_stats['REB_mean']
player_traditional_stats['AST_cv'] = player_traditional_stats['AST_std'] / player_traditional_stats['AST_mean']
player_traditional_stats['PLUS_MINUS_cv'] = player_traditional_stats['PLUS_MINUS_std'] / player_traditional_stats['PLUS_MINUS_mean'].abs()

# Handle infinite values from division by zero
player_traditional_stats = player_traditional_stats.replace([np.inf, -np.inf], np.nan)
player_traditional_stats = player_traditional_stats.fillna(0)

# Merge with stability metrics
player_full_stats = pd.merge(
    player_analysis,
    player_traditional_stats,
    on=['Player_ID', 'player_name'],
    how='inner'
)

# Display the merged data
player_full_stats.head()

In [ ]:
# Calculate correlation between stability metrics and traditional stats
correlation_columns = [
    'system_stability', 'performance_entropy', 'pts_volatility', 'max_eigenvalue', 'max_lyapunov',
    'PTS_mean', 'REB_mean', 'AST_mean', 'STL_mean', 'BLK_mean', 'TOV_mean', 'PLUS_MINUS_mean',
    'PTS_cv', 'REB_cv', 'AST_cv', 'PLUS_MINUS_cv'
]

correlation_matrix = player_full_stats[correlation_columns].corr()

# Plot correlation heatmap
plt.figure(figsize=(14, 12))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f', linewidths=0.5)
plt.title('Correlation: Stability Metrics vs. Traditional Stats', fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
# Focus on correlations with system stability
stability_correlations = correlation_matrix['system_stability'].sort_values(ascending=False)
stability_correlations

In [ ]:
# Visualize correlations with system stability
plt.figure(figsize=(10, 8))
stability_correlations.drop('system_stability').plot(kind='bar')
plt.axhline(y=0, color='black', linestyle='-', alpha=0.3)
plt.title('Correlation with System Stability', fontsize=14)
plt.ylabel('Correlation Coefficient', fontsize=12)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

### Insights from Correlation Analysis

Our correlation analysis reveals several important insights:

1. **System Stability vs. Traditional Volatility Metrics**:
   - System stability shows strong positive correlation with traditional coefficient of variation metrics
   - However, system stability captures additional dimensions of performance dynamics not reflected in simple variance metrics

2. **Stability and Performance Level**:
   - [Describe the relationship between stability and performance level based on your correlation results]
   - This suggests that [interpretation based on your data]

3. **Stability and Playing Style**:
   - [Describe any correlations between stability and playing style indicators like assists, rebounds, etc.]
   - This indicates that [interpretation based on your data]

4. **Entropy vs. Traditional Metrics**:
   - [Describe correlations between performance entropy and traditional stats]
   - This demonstrates that [interpretation based on your data]

These findings highlight the value of our dynamical systems approach in providing insights that go beyond traditional statistical analysis.

## Traditional vs. Dynamics-Based Player Evaluation

Let's compare traditional player evaluation metrics with our dynamics-based approach to identify players who may be undervalued or overvalued by conventional statistics.

In [ ]:
# Calculate traditional and dynamics-based player ratings
player_full_stats['traditional_rating'] = (player_full_stats['PTS_mean'] * 0.6 + 
                                          player_full_stats['PLUS_MINUS_mean'] * 0.4)

player_full_stats['dynamics_rating'] = (player_full_stats['PTS_mean'] * 0.4 + 
                                       player_full_stats['PLUS_MINUS_mean'] * 0.3 +
                                       (10 / (1 + player_full_stats['system_stability'])) * 0.3)

# Calculate rating difference
player_full_stats['rating_difference'] = player_full_stats['dynamics_rating'] - player_full_stats['traditional_rating']

# Normalize ratings for visualization
max_trad = player_full_stats['traditional_rating'].max()
max_dyn = player_full_stats['dynamics_rating'].max()
player_full_stats['traditional_normalized'] = player_full_stats['traditional_rating'] / max_trad
player_full_stats['dynamics_normalized'] = player_full_stats['dynamics_rating'] / max_dyn

# Display players with the biggest rating differences
print("Players most undervalued by traditional metrics:")
undervalued = player_full_stats.nlargest(10, 'rating_difference')
print(undervalued[['player_name', 'traditional_rating', 'dynamics_rating', 'rating_difference', 'system_stability', 'quadrant']])

print("\nPlayers most overvalued by traditional metrics:")
overvalued = player_full_stats.nsmallest(10, 'rating_difference')
print(overvalued[['player_name', 'traditional_rating', 'dynamics_rating', 'rating_difference', 'system_stability', 'quadrant']])

In [ ]:
# Visualize traditional vs. dynamics-based ratings
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 8))

# Players undervalued by traditional metrics
x = np.arange(len(undervalued))
width = 0.35

ax1.bar(x - width/2, undervalued['traditional_normalized'], width, 
       label='Traditional Rating', color='#3498db')
ax1.bar(x + width/2, undervalued['dynamics_normalized'], width,
       label='Dynamics Rating', color='#e74c3c')

ax1.set_xticks(x)
ax1.set_xticklabels(undervalued['player_name'], rotation=45, ha='right')
ax1.legend()
ax1.set_title('Players Undervalued by Traditional Metrics', fontsize=14)
ax1.set_ylim(0, 1.2)

# Players overvalued by traditional metrics
ax2.bar(x - width/2, overvalued['traditional_normalized'], width, 
       label='Traditional Rating', color='#3498db')
ax2.bar(x + width/2, overvalued['dynamics_normalized'], width,
       label='Dynamics Rating', color='#e74c3c')

ax2.set_xticks(x)
ax2.set_xticklabels(overvalued['player_name'], rotation=45, ha='right')
ax2.legend()
ax2.set_title('Players Overvalued by Traditional Metrics', fontsize=14)
ax2.set_ylim(0, 1.2)

plt.tight_layout()
plt.show()

### Market Inefficiency Opportunities

Our comparison of traditional and dynamics-based player evaluation reveals potential market inefficiencies that teams could exploit:

1. **Undervalued Players**:
   - Players with high stability and positive impact who may be undervalued in the market
   - These players offer good value for their expected contribution
   - Examples: [List examples from your undervalued players]

2. **Overvalued Players**:
   - Players with high volatility or negative impact who may be overvalued in the market
   - These players may not provide consistent value commensurate with their perceived worth
   - Examples: [List examples from your overvalued players]

3. **Roster Construction Implications**:
   - Teams should prioritize high-value stability players for core positions
   - High-ceiling volatility players should be added selectively for specific roles
   - High-risk variability players should be avoided unless available at significant discount
   - Low-impact consistency players can provide value in specific role player situations

## Stability Uncertainty Analysis

To ensure the robustness of our stability metrics, let's perform a bootstrap analysis to estimate confidence intervals.

In [ ]:
# Perform bootstrap analysis for a sample player
# Select a player with enough games
sample_player_name = case_study_players.get('High-Value Stability', list(case_study_players.values())[0])
sample_player_id = player_analysis[player_analysis['player_name'] == sample_player_name]['player_id'].iloc[0]
sample_player_data = player_temporal_df[player_temporal_df['Player_ID'] == sample_player_id].copy()

# Number of bootstrap samples
n_bootstrap = 100
bootstrap_results = []

for i in range(n_bootstrap):
    # Resample with replacement
    bootstrap_sample = resample(sample_player_data, replace=True, n_samples=len(sample_player_data))
    
    # Calculate VAR model
    var_results, eigenvalues, lyapunov_exponents = calculate_var_model(bootstrap_sample)
    
    if var_results is not None and eigenvalues is not None:
        # Calculate stability metrics
        max_eigenvalue = np.max(np.abs(eigenvalues))
        max_lyapunov = np.max(lyapunov_exponents)
        
        # Calculate points volatility
        pts_volatility = bootstrap_sample['PTS'].std() / bootstrap_sample['PTS'].mean() if bootstrap_sample['PTS'].mean() > 0 else 0
        
        # Calculate performance entropy
        pts_normalized = (bootstrap_sample['PTS'] - bootstrap_sample['PTS'].min()) / (bootstrap_sample['PTS'].max() - bootstrap_sample['PTS'].min() + 1e-10)
        pts_bins = np.histogram(pts_normalized, bins=10)[0]
        pts_probs = pts_bins / len(bootstrap_sample)
        performance_entropy = -np.sum(pts_probs * np.log2(pts_probs + 1e-10))
        
        # Calculate system stability
        system_stability = 0.3 * max_eigenvalue + 0.3 * (max_lyapunov if max_lyapunov > 0 else 0) + 0.2 * pts_volatility + 0.2 * (performance_entropy / 3)
        
        bootstrap_results.append({
            'max_eigenvalue': max_eigenvalue,
            'max_lyapunov': max_lyapunov,
            'pts_volatility': pts_volatility,
            'performance_entropy': performance_entropy,
            'system_stability': system_stability
        })

# Convert to dataframe
bootstrap_df = pd.DataFrame(bootstrap_results)

# Calculate confidence intervals
confidence_intervals = {}
for column in bootstrap_df.columns:
    lower = np.percentile(bootstrap_df[column], 2.5)
    upper = np.percentile(bootstrap_df[column], 97.5)
    confidence_intervals[column] = (lower, upper)

# Display results
print(f"Bootstrap Analysis for {sample_player_name} (95% Confidence Intervals):")
for metric, (lower, upper) in confidence_intervals.items():
    print(f"{metric}: {lower:.3f} - {upper:.3f}")

In [ ]:
# Visualize the bootstrap distribution of system stability
plt.figure(figsize=(10, 6))
plt.hist(bootstrap_df['system_stability'], bins=20, alpha=0.7, color='skyblue', edgecolor='black')
plt.axvline(x=confidence_intervals['system_stability'][0], color='red', linestyle='--', label='95% CI Lower')
plt.axvline(x=confidence_intervals['system_stability'][1], color='red', linestyle='--', label='95% CI Upper')
plt.axvline(x=bootstrap_df['system_stability'].mean(), color='green', linestyle='-', label='Mean')
plt.xlabel('System Stability', fontsize=12)
plt.ylabel('Frequency', fontsize=12)
plt.title(f'Bootstrap Distribution of System Stability for {sample_player_name}', fontsize=14)
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

### Limitations of the Approach

While our dynamical systems approach provides valuable insights, it's important to acknowledge its limitations:

1. **Sample Size**: Players with fewer games have less reliable stability metrics.
2. **Model Simplicity**: Our VAR(1) model captures only first-order dependencies and may miss more complex patterns.
3. **External Factors**: Our model doesn't account for factors like injuries, team changes, or opponent quality.
4. **Temporal Resolution**: Game-to-game analysis may miss within-game dynamics.
5. **Limited State Space**: We focus on points and plus/minus, but other metrics could provide additional insights.

Despite these limitations, our approach offers a novel perspective on player performance that complements traditional analytics.

## Performance Decomposition

Let's decompose player performance into intrinsic skill, momentum, and contextual factors.

In [ ]:
# Select a player for performance decomposition
decomp_player_name = case_study_players.get('High-Value Stability', list(case_study_players.values())[0])
decomp_player_id = player_analysis[player_analysis['player_name'] == decomp_player_name]['player_id'].iloc[0]
decomp_player_data = player_temporal_df[player_temporal_df['Player_ID'] == decomp_player_id].copy()
decomp_player_data = decomp_player_data.sort_values('GAME_DATE')

# Calculate baseline (intrinsic skill)
baseline_pts = decomp_player_data['PTS'].mean()

# Calculate momentum component
decomp_player_data['momentum_component'] = decomp_player_data['PTS_Momentum']
decomp_player_data['momentum_component'] = decomp_player_data['momentum_component'].fillna(0)

# Calculate contextual component (deviation from baseline not explained by momentum)
decomp_player_data['contextual_component'] = decomp_player_data['PTS'] - baseline_pts - decomp_player_data['momentum_component']

# Calculate component percentages
decomp_player_data['baseline_pct'] = np.abs(baseline_pts) / decomp_player_data['PTS'] * 100
decomp_player_data['momentum_pct'] = np.abs(decomp_player_data['momentum_component']) / decomp_player_data['PTS'] * 100
decomp_player_data['contextual_pct'] = np.abs(decomp_player_data['contextual_component']) / decomp_player_data['PTS'] * 100

# Handle division by zero
decomp_player_data = decomp_player_data.replace([np.inf, -np.inf], np.nan)
decomp_player_data = decomp_player_data.fillna(0)

# Calculate average component breakdown
avg_baseline_pct = decomp_player_data['baseline_pct'].mean()
avg_momentum_pct = decomp_player_data['momentum_pct'].mean()
avg_contextual_pct = decomp_player_data['contextual_pct'].mean()

print(f"Performance Decomposition for {decomp_player_name}:")
print(f"Baseline (Intrinsic Skill): {avg_baseline_pct:.1f}%")
print(f"Momentum: {avg_momentum_pct:.1f}%")
print(f"Contextual Factors: {avg_contextual_pct:.1f}%")

In [ ]:
# Visualize performance decomposition
plt.figure(figsize=(10, 6))

# Create pie chart
labels = ['Intrinsic Skill', 'Momentum', 'Contextual Factors']
sizes = [avg_baseline_pct, avg_momentum_pct, avg_contextual_pct]
colors = ['#3498db', '#2ecc71', '#e74c3c']
explode = (0.1, 0, 0)  # explode the 1st slice (Intrinsic Skill)

plt.pie(sizes, explode=explode, labels=labels, colors=colors,
        autopct='%1.1f%%', shadow=True, startangle=140)
plt.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle
plt.title(f'Performance Decomposition for {decomp_player_name}', fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
# Visualize performance components over time
plt.figure(figsize=(14, 8))

# Create stacked area chart
plt.stackplot(decomp_player_data['GAME_DATE'],
              [baseline_pts] * len(decomp_player_data),
              decomp_player_data['momentum_component'],
              decomp_player_data['contextual_component'],
              labels=['Intrinsic Skill', 'Momentum', 'Contextual Factors'],
              colors=['#3498db', '#2ecc71', '#e74c3c'],
              alpha=0.7)

# Add total points line
plt.plot(decomp_player_data['GAME_DATE'], decomp_player_data['PTS'], 'k-', label='Total Points')

plt.xlabel('Game Date', fontsize=12)
plt.ylabel('Points', fontsize=12)
plt.title(f'Performance Components Over Time for {decomp_player_name}', fontsize=14)
plt.legend(loc='upper left')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

### Performance Component Analysis

Our performance decomposition reveals how different factors contribute to a player's performance:

1. **Intrinsic Skill (Baseline)**:
   - Represents the player's fundamental skill level and average performance
   - Accounts for [X]% of performance on average
   - More stable players have higher baseline contributions

2. **Momentum**:
   - Represents the impact of recent performance trends on current performance
   - Accounts for [Y]% of performance on average
   - More volatile players show stronger momentum effects

3. **Contextual Factors**:
   - Represents game situation, matchups, and other external factors
   - Accounts for [Z]% of performance on average
   - Players with higher entropy show stronger contextual effects

This decomposition provides insights into the sources of player performance and can inform coaching and development strategies.

## Conclusion

In this notebook, we've built on the dynamical systems analysis from the previous notebook to classify players into distinct stability profiles, examine case studies of different player types, and translate our mathematical findings into practical basketball insights.

Key accomplishments:
1. Classified players into stability categories and quadrants based on their performance dynamics
2. Analyzed case studies of players with different stability profiles
3. Translated mathematical findings into basketball insights and strategic applications
4. Compared traditional and dynamics-based player evaluation to identify market inefficiencies
5. Performed uncertainty analysis to assess the robustness of our stability metrics
6. Decomposed player performance into intrinsic skill, momentum, and contextual factors

Our analysis demonstrates the value of applying dynamical systems theory to basketball analytics, providing insights that go beyond traditional statistical approaches. These insights can inform player acquisition, lineup construction, game strategy, and player development decisions, potentially giving teams a competitive advantage.

In the next notebook, we'll explore team playing styles and how they relate to player stability profiles.